In [ ]:
%%bash
wget https://vina.scripps.edu/wp-content/uploads/sites/55/2020/12/autodock_vina_1_1_2_linux_x86.tgz
tar xzvf autodock_vina_1_1_2_linux_x86.tgz

autodock_vina_1_1_2_linux_x86/
autodock_vina_1_1_2_linux_x86/LICENSE
autodock_vina_1_1_2_linux_x86/bin/
autodock_vina_1_1_2_linux_x86/bin/vina
autodock_vina_1_1_2_linux_x86/bin/vina_split


--2022-11-10 22:35:39--  https://vina.scripps.edu/wp-content/uploads/sites/55/2020/12/autodock_vina_1_1_2_linux_x86.tgz
Resolving vina.scripps.edu (vina.scripps.edu)... 192.26.252.19
Connecting to vina.scripps.edu (vina.scripps.edu)|192.26.252.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1238242 (1.2M) [application/x-gzip]
Saving to: ‘autodock_vina_1_1_2_linux_x86.tgz.1’

     0K .......... .......... .......... .......... ..........  4%  516K 2s
    50K .......... .......... .......... .......... ..........  8% 1.02M 2s
   100K .......... .......... .......... .......... .......... 12% 1.02M 1s
   150K .......... .......... .......... .......... .......... 16%  118M 1s
   200K .......... .......... .......... .......... .......... 20%  336M 1s
   250K .......... .......... .......... .......... .......... 24% 1.03M 1s
   300K .......... .......... .......... .......... .......... 28%  180M 1s
   350K .......... .......... .......... .......... .........

In [ ]:
alias vina /content/autodock_vina_1_1_2_linux_x86/bin/vina

In [ ]:
!wget https://static-bcrf.biochem.wisc.edu/tutorials/autodock/protein.pdbqt_.txt

--2022-11-10 22:35:41--  https://static-bcrf.biochem.wisc.edu/tutorials/autodock/protein.pdbqt_.txt
Resolving static-bcrf.biochem.wisc.edu (static-bcrf.biochem.wisc.edu)... 128.104.80.63
Connecting to static-bcrf.biochem.wisc.edu (static-bcrf.biochem.wisc.edu)|128.104.80.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 216232 (211K) [text/plain]
Saving to: ‘protein.pdbqt_.txt.1’

protein.pdbqt_.txt. 100%[===================>] 211.16K  --.-KB/s    in 0.1s    

2022-11-10 22:35:42 (2.07 MB/s) - ‘protein.pdbqt_.txt.1’ saved [216232/216232]



In [ ]:
!wget  https://static-bcrf.biochem.wisc.edu/tutorials/autodock/ligand.pdbqt_.txt

--2022-11-10 22:35:46--  https://static-bcrf.biochem.wisc.edu/tutorials/autodock/ligand.pdbqt_.txt
Resolving static-bcrf.biochem.wisc.edu (static-bcrf.biochem.wisc.edu)... 128.104.80.63
Connecting to static-bcrf.biochem.wisc.edu (static-bcrf.biochem.wisc.edu)|128.104.80.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3878 (3.8K) [text/plain]
Saving to: ‘ligand.pdbqt_.txt.1’

ligand.pdbqt_.txt.1 100%[===================>]   3.79K  --.-KB/s    in 0s      

2022-11-10 22:35:46 (249 MB/s) - ‘ligand.pdbqt_.txt.1’ saved [3878/3878]



In [ ]:
pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.5 MB 1.1 MB/s 


In [ ]:
#NOT WRITTEN BY SELF - Sourced from official RDKit Tutorials
import sys
import argparse
import pandas as pd
from rdkit import Chem, RDLogger
import rdkit.Chem.PropertyMol
import os

RDLogger.DisableLog('rdApp.*')


def convert_smi_to_mol(smi_list):
    # convert our smiles to mols
    smiles = pd.read_csv(smi_list, header=None)
    mol_list = []
    for i in range(smiles.shape[0]):
        smi = smiles[0][i]
        mol = Chem.MolFromSmiles(smi)
        if mol is not None:
          mol_list.append(mol)
    return mol_list


def mol_props(mols, ext):
    # add the name property to each mol and create a pandas dataframe associating the names to the smiles
    i = 1
    mols_to_export, smile_export = [], []
    for mol in mols:
        name = str(ext) + '-' + str(i)
        pm = Chem.PropertyMol.PropertyMol(mol)
        pm.SetProp('Name', name)
        mols_to_export.append(pm)

        mol_dict = {}
        mol_dict['Name'] = name
        smile = Chem.MolToSmiles(mol)
        mol_dict['SMILE'] = smile
        smile_export.append(mol_dict)
        i += 1

    df = pd.DataFrame(smile_export)
    return df, mols_to_export


def write_sdf(mols, output):
    # convert our mols to .sdf files
    sdf_dir = '{0}/'.format(output)
    os.mkdir(sdf_dir)
    for m in mols:
        pm = Chem.PropertyMol.PropertyMol(m)
        name = pm.GetProp('Name')
        w = Chem.SDWriter(str(sdf_dir) + str(name) + '.sdf')
        w.write(m)


def smi_to_sdf(smi_list, ext, df_name, output):
    # convert smiles to mols
    mols = convert_smi_to_mol(smi_list)

    # add the name property to mols
    smiles_df, mol_list = mol_props(mols, str(ext))

    # save the name-smile dataframe to excel
    smiles_df.to_excel(str(df_name) + '.xlsx')

    # create the .sdf files fromm our mols
    write_sdf(mol_list, output)
    print('Done creating SDF files')

In [ ]:
#smi_to_sdf("/content/generated_smiles_nextgen_PDGFRAGen1.csv", "Gen1", "Gen1 Ligands", "format")

In [ ]:
#!zip -r /content/gen1_ligands.zip /content/gen1_ligands-sdf

In [ ]:
# from google.colab import files
# files.download("/content/file.zip")

In [ ]:
pip install openbabel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 82 kB 1.1 MB/s 
  ERROR: Failed building wheel for openbabel
  Running setup.py clean for openbabel
Failed to build openbabel
    Running setup.py install for openbabel ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-7mth34us/openbabel_35bf1d27a51243a1bfd9931a2f7ceb39/setup.py'"'"'; __file__='"'"'/tmp/pip-install-7mth34us/openbabel_35bf1d27a51243a1bfd9931a2f7ceb39/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-t79gz637/install-record.txt --single-version-externally-managed --compile --i

In [ ]:
!unzip /content/gen1_ligands.zip

Archive:  /content/gen1_ligands.zip
   creating: content/gen1_ligands-sdf/
  inflating: content/gen1_ligands-sdf/Gen1-4.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-252.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-99.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-307.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-163.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-44.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-85.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-91.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-118.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-119.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-121.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-258.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-201.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-131.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-96.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-214.sdf  
  inflating: content/gen1_ligands-sdf/Gen1-41.sdf  
  inflating: content/gen1_ligand

In [ ]:
#NOT WRITTEN BY SELF - Sourced from official Babel tutorials
import os
import sys
import babel

def sdf_to_pdbqt():
    os.chdir("/content/content/gen1_ligands-sdf")
    #with open(os.path.join("/content/content/all_folder", filename), 'r') as f: # open in readonly mode
    # batch converts the .sdf files to .pdb files and adds hydrogens then
    # converts the .pdb files to .pdbqt files while preserving the hydrogens
    # finally, removes the interim files
    os.system('obabel -isdf /content/content/gen1_ligands-sdf/*.sdf -opdb /content/content/gen1_ligands-sdf/*.pdb  -h')
    os.system('obabel -ipdb /content/content/gen1_ligands-sdf/*.pdb -opdbqt /content/content/gen1_ligands-sdf/*.pdbqt -xh -m')
    print('Ligand Conversions Complete!')

In [ ]:
sdf_to_pdbqt()

Ligand Conversions Complete!


In [ ]:
!unzip /content/gen1_ligands_pdbqt.zip

Archive:  /content/gen1_ligands_pdbqt.zip
  inflating: gen1_ligands_pdbqt/Gen1-1.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-10.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-100.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-101.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-102.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-103.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-104.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-105.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-106.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-107.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-108.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-109.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-11.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-110.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-111.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-112.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-113.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-114.pdbqt  
  inflating: gen1_ligands_pdbqt/Gen1-115.pdbqt  
  inflating: gen1_ligands_pdbqt

In [ ]:
pip install vina

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.0 MB 5.1 MB/s 


In [ ]:
#library
from vina import Vina
import os
v= Vina(sf_name = 'vina')
v.set_receptor('/content/protein.pdbqt')
v.compute_vina_maps(center=[-2.27, -10.88, 24.23], box_size=[30, 30, 30])

In [ ]:
os.chdir('/content')

In [ ]:
%%bash
cat > conf.txt <<EOF

EOF

In [ ]:
for i in range(1,311):
  conf = open("/content/conf.txt", "w")
  info = """
  receptor = /content/protein.pdbqt
  ligand = /content/gen1_ligands_pdbqt/Gen1-{}.pdbqt

  out = /content/all_folder/all{}.pdbqt

  center_x = -2.27
  center_y = -11
  center_z = 24

  size_x = 30
  size_y = 30
  size_z = 30
""".format(str(i),str(i))
  conf.write(info)
  conf.close()
  myFile = open("/content/conf.txt", "r")

  print(myFile.read())
  %vina --config conf.txt --log log.txt --exhaustiveness 32


Streaming output truncated to the last 5000 lines.
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 40 CPUs
Reading input ... done.
Setting up the scoring function ... done.
Analyzing the binding site ... done.
Using random seed: -1381137956
Performing search ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
done.
Refining results ... done.

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1         -8.4      0.000      0.000
   2         -8.4      3.089      4.483
   3         -8.2      3.239      8.329
   4         -8.2      3.111      4.380
   5         -8.2      2.764      4.129
   6         -8.2      3.269      7.628
   7         -8.1    

In [ ]:
!zip -r /content/all_folder.zip /content/all_folder

  adding: content/all_folder/ (stored 0%)
  adding: content/all_folder/all149.pdbqt (deflated 85%)
  adding: content/all_folder/all81.pdbqt (deflated 84%)
  adding: content/all_folder/all231.pdbqt (deflated 84%)
  adding: content/all_folder/all119.pdbqt (deflated 85%)
  adding: content/all_folder/all177.pdbqt (deflated 85%)
  adding: content/all_folder/all118.pdbqt (deflated 85%)
  adding: content/all_folder/all186.pdbqt (deflated 85%)
  adding: content/all_folder/all108.pdbqt (deflated 84%)
  adding: content/all_folder/all248.pdbqt (deflated 85%)
  adding: content/all_folder/all153.pdbqt (deflated 84%)
  adding: content/all_folder/all139.pdbqt (deflated 85%)
  adding: content/all_folder/all113.pdbqt (deflated 84%)
  adding: content/all_folder/all33.pdbqt (deflated 85%)
  adding: content/all_folder/all215.pdbqt (deflated 85%)
  adding: content/all_folder/all68.pdbqt (deflated 85%)
  adding: content/all_folder/all151.pdbqt (deflated 84%)
  adding: content/all_folder/all277.pdbqt (deflat

In [ ]:
from google.colab import files
files.download("/content/all_folder.zip")

In [ ]:
!apt-get -qq install -y openbabel

Selecting previously unselected package libopenbabel4v5.
(Reading database ... 123941 files and directories currently installed.)
Preparing to unpack .../libopenbabel4v5_2.3.2+dfsg-3build1_amd64.deb ...
Unpacking libopenbabel4v5 (2.3.2+dfsg-3build1) ...
Selecting previously unselected package openbabel.
Preparing to unpack .../openbabel_2.3.2+dfsg-3build1_amd64.deb ...
Unpacking openbabel (2.3.2+dfsg-3build1) ...
Setting up libopenbabel4v5 (2.3.2+dfsg-3build1) ...
Setting up openbabel (2.3.2+dfsg-3build1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.6) ...


In [ ]:
!babel -ipdbqt 'protein.pdbqt' -opdb 'protein.pdb'

1 molecule converted
10 info messages 16 audit log messages 


In [ ]:
!babel -ipdbqt 'all.pdbqt' -opdb 'all.pdb'

*** Open Babel Error  in OpenAndSetFormat
  Cannot open all.pdbqt
0 molecules converted
1 errors 


In [ ]:
!babel -ipdbqt /content/all_folder/*.pdbqt -opdb /content/all_folder/*.pdb

2114 molecules converted
237 files output. The first is /content/all_folder/all100.pdb
162 info messages 70545 audit log messages 


In [ ]:
os.chdir("/content/")
!unzip /content/all_folder.zip

Archive:  /content/all_folder.zip
   creating: content/all_folder/
  inflating: content/all_folder/all149.pdbqt  
  inflating: content/all_folder/all81.pdbqt  
  inflating: content/all_folder/all231.pdbqt  
  inflating: content/all_folder/all119.pdbqt  
  inflating: content/all_folder/all177.pdbqt  
  inflating: content/all_folder/all118.pdbqt  
  inflating: content/all_folder/all186.pdbqt  
  inflating: content/all_folder/all108.pdbqt  
  inflating: content/all_folder/all248.pdbqt  
  inflating: content/all_folder/all153.pdbqt  
  inflating: content/all_folder/all139.pdbqt  
  inflating: content/all_folder/all113.pdbqt  
  inflating: content/all_folder/all33.pdbqt  
  inflating: content/all_folder/all215.pdbqt  
  inflating: content/all_folder/all68.pdbqt  
  inflating: content/all_folder/all151.pdbqt  
  inflating: content/all_folder/all277.pdbqt  
  inflating: content/all_folder/all298.pdbqt  
  inflating: content/all_folder/all50.pdbqt  
  inflating: content/all_folder/all20.pdbqt 

In [ ]:
import os
vina_scores_map = {}
for filename in os.listdir("/content/all_folder"):

#with open(os.path.join("/content/content/all_folder", filename), 'r') as f: # open in readonly mode
  if filename[-3:] == "pdb":
    all_file = open("/content/all_folder/" + filename, "r")
    vina_binding_score = float(all_file.read().split()[7])
    vina_scores_map[filename] = vina_binding_score

#print(vina_scores_map)
{k: v for k, v in sorted(vina_scores_map.items(), key=lambda item: item[1])}

{'all205.pdb': -12.9,
 'all99.pdb': -12.7,
 'all98.pdb': -12.6,
 'all41.pdb': -12.6,
 'all188.pdb': -12.4,
 'all84.pdb': -12.2,
 'all42.pdb': -12.1,
 'all212.pdb': -12.0,
 'all120.pdb': -11.9,
 'all37.pdb': -11.9,
 'all20.pdb': -11.8,
 'all117.pdb': -11.8,
 'all231.pdb': -11.8,
 'all105.pdb': -11.7,
 'all218.pdb': -11.6,
 'all125.pdb': -11.6,
 'all287.pdb': -11.4,
 'all240.pdb': -11.3,
 'all63.pdb': -11.2,
 'all29.pdb': -11.2,
 'all104.pdb': -11.1,
 'all115.pdb': -11.0,
 'all310.pdb': -10.9,
 'all290.pdb': -10.9,
 'all176.pdb': -10.9,
 'all147.pdb': -10.8,
 'all116.pdb': -10.8,
 'all95.pdb': -10.7,
 'all185.pdb': -10.6,
 'all226.pdb': -10.6,
 'all43.pdb': -10.6,
 'all96.pdb': -10.6,
 'all83.pdb': -10.6,
 'all56.pdb': -10.5,
 'all186.pdb': -10.5,
 'all108.pdb': -10.5,
 'all89.pdb': -10.4,
 'all213.pdb': -10.4,
 'all74.pdb': -10.4,
 'all129.pdb': -10.4,
 'all80.pdb': -10.4,
 'all24.pdb': -10.4,
 'all103.pdb': -10.3,
 'all202.pdb': -10.3,
 'all156.pdb': -10.3,
 'all143.pdb': -10.3,
 'all5